# Notebook 03: Data Preprocessing

Data cleaning and preparation for model training.\
Handles: missing values, outliers, type conversions.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import warnings
warnings.filterwarnings('ignore')

ames_data = pd.read_csv('data/ames/AmesHousing.csv')
malaysia_data = pd.read_csv('data/malaysia/malaysia_house_price_data_2025.csv')

print(f"Ames: {ames_data.shape} | Malaysia: {malaysia_data.shape}")

Ames: (2930, 82) | Malaysia: (2000, 8)


## Ames Preprocessing

In [2]:
# Create a copy
ames_clean = ames_data.copy()

# Handle missing values with domain knowledge
fill_none_cols = ['Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
                  'BsmtFinType2', 'Electrical', 'FireplaceQu', 'GarageType', 
                  'GarageFinish', 'GarageQual', 'GarageCond', 'PoolQC', 'Fence', 'MiscFeature']

for col in fill_none_cols:
    if col in ames_clean.columns and ames_clean[col].isnull().sum() > 0:
        ames_clean[col] = ames_clean[col].fillna('None')

# Fill numerical missing values with median
numerical_cols = ames_clean.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if ames_clean[col].isnull().sum() > 0:
        ames_clean[col] = ames_clean[col].fillna(ames_clean[col].median())

print(f"Ames missing values after cleaning: {ames_clean.isnull().sum().sum()}")
print(f"Ames shape: {ames_clean.shape}")

Ames missing values after cleaning: 9976
Ames shape: (2930, 82)


## Malaysia Preprocessing

In [3]:
# Create a copy
malaysia_clean = malaysia_data.copy()

# Fill numerical missing values
numerical_cols = malaysia_clean.select_dtypes(include=[np.number]).columns
for col in numerical_cols:
    if malaysia_clean[col].isnull().sum() > 0:
        malaysia_clean[col] = malaysia_clean[col].fillna(malaysia_clean[col].median())

print(f"Malaysia missing values after cleaning: {malaysia_clean.isnull().sum().sum()}")
print(f"Malaysia shape: {malaysia_clean.shape}")

Malaysia missing values after cleaning: 0
Malaysia shape: (2000, 8)


## Outlier Detection

In [4]:
# Identify outliers in Ames using IQR
Q1 = ames_clean['SalePrice'].quantile(0.25)
Q3 = ames_clean['SalePrice'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

outliers_ames = (ames_clean['SalePrice'] < lower_bound) | (ames_clean['SalePrice'] > upper_bound)
print(f"Ames outliers detected: {outliers_ames.sum()}")

# Identify outliers in Malaysia
Q1_my = malaysia_clean['Median_Price'].quantile(0.25)
Q3_my = malaysia_clean['Median_Price'].quantile(0.75)
IQR_my = Q3_my - Q1_my
lower_bound_my = Q1_my - 1.5 * IQR_my
upper_bound_my = Q3_my + 1.5 * IQR_my

outliers_malaysia = (malaysia_clean['Median_Price'] < lower_bound_my) | (malaysia_clean['Median_Price'] > upper_bound_my)
print(f"Malaysia outliers detected: {outliers_malaysia.sum()}")

# Flag (don't remove yet - for analysis)
ames_clean['Is_Outlier'] = outliers_ames
malaysia_clean['Is_Outlier'] = outliers_malaysia

Ames outliers detected: 137
Malaysia outliers detected: 123


Next cell deletes all outliers and drops `Is_Outlier` columns from both datasets.\
If needed, **comment** the cell to prevent deleting of examples.

In [5]:
# Remove rows flagged as outliers and drop the 'Is_Outlier' column
print(f"Ames outliers to remove: {ames_clean['Is_Outlier'].sum()}")
print(f"Malaysia outliers to remove: {malaysia_clean['Is_Outlier'].sum()}")

ames_clean = ames_clean.loc[~ames_clean['Is_Outlier']].reset_index(drop=True)
malaysia_clean = malaysia_clean.loc[~malaysia_clean['Is_Outlier']].reset_index(drop=True)

# Drop the flag column
if 'Is_Outlier' in ames_clean.columns:
    ames_clean.drop(columns='Is_Outlier', inplace=True)
if 'Is_Outlier' in malaysia_clean.columns:
    malaysia_clean.drop(columns='Is_Outlier', inplace=True)

print(f"Ames shape after removal: {ames_clean.shape}")
print(f"Malaysia shape after removal: {malaysia_clean.shape}")

Ames outliers to remove: 137
Malaysia outliers to remove: 123
Ames shape after removal: (2793, 82)
Malaysia shape after removal: (1877, 8)


**Note**: on the future steps I found out, that `Is_Outlier` feature is recognised by XGBoost as the main feature, that is incorrect.\
So, deleting those columns from both datasets is required at this stage.

## Data Type Conversion

In [6]:
# Ames: Convert categorical to category type for memory efficiency
categorical_cols_ames = ames_clean.select_dtypes(include=['object']).columns
for col in categorical_cols_ames:
    ames_clean[col] = ames_clean[col].astype('category')

# Malaysia: Same for categorical
categorical_cols_malaysia = malaysia_clean.select_dtypes(include=['object']).columns
for col in categorical_cols_malaysia:
    if col in malaysia_clean.columns:
        malaysia_clean[col] = malaysia_clean[col].astype('category')

print("Data types converted to category for efficiency")
print(f"Ames memory usage: {ames_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print(f"Malaysia memory usage: {malaysia_clean.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

Data types converted to category for efficiency
Ames memory usage: 0.97 MB
Malaysia memory usage: 0.28 MB


## Save Cleaned Data

In [ ]:
# Save cleaned datasets
ames_clean.to_csv('data/processed/ames_cleaned.csv', index=False)
malaysia_clean.to_csv('data/processed/malaysia_cleaned.csv', index=False)

print("Cleaned data saved to data/processed/")
print(f"  - ames_cleaned.csv")
print(f"  - malaysia_cleaned.csv")

Cleaned data saved to data/processed/
  - ames_cleaned.csv
  - malaysia_cleaned.csv


## Preprocessing Summary

In [8]:
print("PREPROCESSING SUMMARY")
print(f"\nAmes Dataset:")
print(f"  Original shape: {ames_data.shape}")
print(f"  Cleaned shape: {ames_clean.shape}")
print(f"  Missing values: {ames_clean.isnull().sum().sum()}")
print(f"  Outliers flagged: {outliers_ames.sum()}")

print(f"\nMalaysia Dataset:")
print(f"  Original shape: {malaysia_data.shape}")
print(f"  Cleaned shape: {malaysia_clean.shape}")
print(f"  Missing values: {malaysia_clean.isnull().sum().sum()}")
print(f"  Outliers flagged: {outliers_malaysia.sum()}")

PREPROCESSING SUMMARY

Ames Dataset:
  Original shape: (2930, 82)
  Cleaned shape: (2793, 82)
  Missing values: 9681
  Outliers flagged: 137

Malaysia Dataset:
  Original shape: (2000, 8)
  Cleaned shape: (1877, 8)
  Missing values: 0
  Outliers flagged: 123
